# Exercise 7 - Process Tasks in Order of Completion

**GOAL:** The goal of this exercise is to show how to use `ray.wait` to process tasks in the order that they finish.

See the documentation for ray.wait at https://ray.readthedocs.io/en/latest/api.html#ray.wait.

## Concepts for this exercise - `ray.wait`

After launching a number of tasks, you may want to run the results sequentially. To do so, we build off of exercise 6 and use `ray.wait` to execute the results sequentially. 

We are able to use `ray.wait` because the two lists returned by **`ray.wait` maintains the ordering of the input list**. That is, if `f` is a remote function, the code 
```python
    results = ray.wait([f.remote(i) for i in range(100)], num_results=10)
```
will return `(ready_list, remain_list)` and the `ObjectID`s of in those lists will be ordered by the argument passed to `f` above.

In [1]:
import os
os.system('pip install ray')

0

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

In [3]:
ray.init(num_cpus=5, include_webui=False, ignore_reinit_error=True)

2019-05-16 11:54:25,083	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-16 11:54:25,087	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-16_11-54-25_126/logs.
2019-05-16 11:54:25,206	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:20636 to respond...
2019-05-16 11:54:25,338	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:61669 to respond...
2019-05-16 11:54:25,341	INFO services.py:804 -- Starting Redis shard with 2.58 GB max memory.
2019-05-16 11:54:25,384	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-16_11-54-25_126/logs.
2019-05-16 11:54:25,387	INFO services.py:1427 -- Starting the Plasma object store with 3.87 GB memory using /dev/shm.


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2019-05-16_11-54-25_126/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-16_11-54-25_126/sockets/raylet',
 'redis_address': '172.28.0.2:20636',
 'webui_url': None}

In [0]:
@ray.remote
def f():
    time.sleep(np.random.uniform(0, 5))
    return time.time()

**EXERCISE:** Change the code below to use `ray.wait` to get the results of the tasks in the order that they complete.

**NOTE:** It would be a simple modification to maintain a pool of 10 experiments and to start a new experiment whenever one finishes.

In [17]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

result_ids = [f.remote() for _ in range(10)]
                                    

# Get the results.
results = []
while True:
   completed_ids, remaining_ids = ray.wait(result_ids, num_returns=1, timeout=None)
   result_ids =  remaining_ids
   result = ray.get(completed_ids[0])
   results.append(result)
   print('Processing result which finished after {} seconds.'
          .format(result - start_time))
   if len(result_ids) == 0:
    break
    
end_time = time.time()
duration = end_time - start_time

   




Processing result which finished after 0.10110354423522949 seconds.
Processing result which finished after 1.6244089603424072 seconds.
Processing result which finished after 2.1286706924438477 seconds.
Processing result which finished after 2.2877707481384277 seconds.
Processing result which finished after 2.4664018154144287 seconds.
Processing result which finished after 2.5016775131225586 seconds.
Processing result which finished after 2.8285279273986816 seconds.
Processing result which finished after 6.1847755908966064 seconds.
Processing result which finished after 6.212692975997925 seconds.
Processing result which finished after 6.596499919891357 seconds.


**VERIFY:** Run some checks to verify that the changes you made to the code were correct. Some of the checks should fail when you initially run the cells. After completing the exercises, the checks should pass.

In [18]:
assert results == sorted(results), ('The results were not processed in the '
                                    'order that they finished.')

print('Success! The example took {} seconds.'.format(duration))

Success! The example took 6.598791122436523 seconds.
